In [1]:
%load_ext chime
import os, sys
from datetime import datetime
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.ops import unary_union

In [87]:
adwr = gpd.read_file("./AGIC_NHD_WG.gdb", layer="ADWR_NHDPilotArea").to_crs("epsg:3857").explode()
adeq = gpd.read_file("./AGIC_NHD_WG.gdb", layer="ADEQ_NHDPilotArea").explode().reset_index(drop=True)
pcwash = gpd.read_file("./AGIC_NHD_WG.gdb", layer="PC_NHDPilotArea").explode().reset_index(drop=True)
nhd = gpd.read_file("./AGIC_NHD_WG.gdb", layer="NHD_NHDPilotArea").explode().reset_index(drop=True)

#pc has some bad geometrys that can be fixed with self-intersection
pcwash = gpd.overlay(pcwash, pcwash, how="intersection")

In [88]:
def getEndPointVerticies(geom):
    coords = geom.coords
    begPoint = Point(coords[0])
    endPoint = Point(coords[-1])
    return pd.Series([begPoint, endPoint])

In [89]:
dfs = {"NHD": nhd, "ADEQ": adeq, "ADWR": adwr, "PC": pcwash}

for name, df in dfs.items():
    obpep_file = f"{name}_BPEP.gpkg"
    if os.path.exists(obpep_file):
        continue
    print(f"{name}")
    df[["BegPoint", "EndPoint"]] = df.geometry.apply(lambda g: getEndPointVerticies(g))

    bp = df[["geometry", "BegPoint"]].copy()
    bp["geometry"] = bp["BegPoint"]
    ep = df[["geometry", "EndPoint"]].copy()
    ep["geometry"] = ep["EndPoint"]
    bpep = pd.concat([bp,ep]).reset_index()
    #bpep["EndPoint"], bpep["BegPoint"]
    del df["EndPoint"], df["BegPoint"]
    bpep.geometry = bpep.geometry.apply(lambda g: g.buffer(0.5))

    def getDupPoints(df):
        df["X"] = round(df.geometry.centroid.x)
        df["Y"] = round(df.geometry.centroid.y)
        dfg = df.groupby(["X","Y"]).agg({"X":"count","Y":"count","geometry":"first"}).reset_index(drop=True)
        dfg["count"] = dfg.apply(lambda r: 1 if r["X"] and r["Y"] == 1 else r["X"], axis=1)
        del dfg["X"], dfg["Y"]
        return gpd.GeoDataFrame(dfg,crs=df.crs)

    bpep_g = getDupPoints(bpep)

    bpep_once = bpep_g[bpep_g["count"] == 1].reset_index(drop=True)
    bpep_multi = bpep_g[bpep_g["count"] > 1].reset_index(drop=True)
    df = gpd.sjoin(df, bpep_once, how="left")

    del df["index_right"]
    df.rename(columns=({"count":"Dangle"}), inplace=True)
    df["Dangle"] = df["Dangle"].apply(lambda d: 1 if not pd.isnull(d) else 0)

    df.to_file(f"{name}_Dangles.gpkg", driver="GPKG")
    
    bpep_multi.geometery = bpep_multi.geometry.centroid
    
    bpep_multi.to_file(obpep_file, driver = "GPKG")

    %chime